# Rendu du TP2 de Billy WEYNANS et Sébastien KERBOURC'H

&nbsp;
&nbsp;

### À cause de l'utilisation d'OpenCV, **il est préférable d'ouvrir ce rendu avec un IDE** tel que *PyCharm* ou *VS Code*.
Les fenêtres ouvertes par OpenCV ne se comportent pas correctement lors de l'exécution via ```jupyter notebook file.ipynb```
&nbsp;
&nbsp;
### Pour fermer les fenêtres OpenCV, il faut utiliser q xou échap.


In [1]:
import numpy as np
import cv2
import os
np.seterr(divide='ignore', invalid='ignore')
os.chdir("src")

# Histogramme de couleurs

### Q1 Histogramme Yuv pour détection de changement de plan

In [2]:
cap = cv2.VideoCapture('../videos/Extrait1-Cosmos_Laundromat1(340p).m4v')

# Convention opencv
# Si entier alors entre 0 et 255
# Si float entre 0 et 1

# Check if the video file was successfully loaded
if not cap.isOpened():
    print("Error loading video file")

# Loop through the video frames
while cap.isOpened():
    # Read the next frame from the video file
    ret, frame = cap.read()

    frameYuv = cv2.cvtColor(frame, cv2.COLOR_BGR2YUV)
    # Don't forget the [] around the frame
    hist = cv2.calcHist([frameYuv], [1,2], None, [256, 256], [0,255, 0, 255])

    # Log of histogram
    hist = np.log(hist)

    # Normalization of histogram
    hist = hist/ hist.max() * 255

    # Conversion for opencv
    hist = hist.astype(np.uint8)

    # Colormap because it's cool
    hist = cv2.applyColorMap(hist, cv2.COLORMAP_JET)

    # Check if there are no more frames
    if not ret:
        break

    # Display the frame
    cv2.imshow('frame', frame)
    cv2.imshow('histogramme', hist)

    # Wait for a key press
    if cv2.waitKey(25) & 0xFF == ord('q'):
        break

# Release the video file and close the window
cap.release()
cv2.destroyAllWindows()


On remarque des changements soudains lors des changements de plan alors que les scènes ont des changements progressifs de leur histogramme.

On pourrait calculer l'auto-corrélation entre l'histogramme courant et le précédent.
La valeur de cette auto-corrélation serait ensuite comparée à un seuil prédéfini et lorsque l'auto-corrélation dépasse ce seuil, on détecterait un changement de plan.

# Histogramme de vitesses

### Q2 Méthode de Färneback pour estimation du flux optique

La méthode de Färneback utilise 2 images pour donner une estimation dense du flux optique. Pour chaque image i, un découpage en zone est réalisé et chaque entourage de chaque point est estimée par une expression polynomiale matricielle notée f_i(x). La forme des polynomes étant connue, on peut ainsi remonter au déplacement d (qui est un déplacement sur une zone entière et paramétré à composantes polynomiales) en minimisant une somme d'erreur pondérée d'écarts entre f_2(x) et f_1(x-d) pour chaque point de la zone.
Ainsi, le déplacement d calculé est le déplacement minimisant l'erreur sur une zone entière.

Afin d'affiner l'estimation du flux optique, la méthode est réalisée sur plusieurs résolutions d'images et itérée sur des zones de plus en plus petites pour chaque résolution.

Au début, la résolution est faible (grande échelle) afin d'obtenir un déplacement moins fin mais qui permet de calculer de grands déplacements (donc de grandes vitesses) et qui apportera de la connaissance a priori pour les itérations et niveaux suivants.


Les paramètres de la fonction cv2.calcOpticalFlowFarneback sont :
- prvs : Image 1
- next : Image 2
- pyr_scale : Taux de réduction de l'échelle entre les niveaux de la pyramide d'images (càd la collection d'images créées à partir de l'image initiale en changeant l'échelle)
    - Un pyr_scale trop bas donne des zones bruitées
    - Peu d'amélioration au-delà de 0.5
- levels : Nombre de niveaux de la pyramide
    - Plus levels est petit, moins les résultats seront stables
- winsize : Taille de zone pour le calcul de d
    - L'augmenter rend le flux optique plus grossier
    - Le mettre à 1 fait apparaître les résultats bruités dont Färneback parle dans son papier
- iterations : Nombre d'iterations pour chaque niveau de la pyramide
    - Augmenter iterations donne de meilleurs contours au flux optique
    - Peu d'améliorations au-delà de 3
- poly_n : Taille du voisinage pour un pixel
    - Augmenter poly_n semble rendre le flux optique plus flou donc plus lisse
- poly_sigma : Écart-type du filtre gaussien utilisé pour calculer les dérivées
    - Augmenter poly_sigma réduit la capacité de détection des mouvements rapides
    - Réduire poly_sigma semble donner un flux optique plus homogène
- flags : Options supplémentaires permettant de customiser la fonction

### Q3 Histogramme de vitesse pour détermination de type de plan

Voici le code permettant de calculer et afficher l'histogramme 2D de la probabilité jointe des composantes (Vx, Vy) du flot optique.

In [4]:
# Ouverture du flux video
cap = cv2.VideoCapture("../videos/Extrait5-Matrix-Helicopter_Scene(280p).m4v")
# cap = cv2.VideoCapture("../videos/Rotation_OX(Tilt).m4v")
# cap = cv2.VideoCapture("../videos/Rotation_OY(Pan).m4v")
# cap = cv2.VideoCapture("../videos/Rotation_OZ(Roll).m4v")
# cap = cv2.VideoCapture("../videos/ZOOM_O_TRAVELLING.m4v")
# cap = cv2.VideoCapture("../videos/Travelling_OX.m4v")
# cap = cv2.VideoCapture("../videos/Travelling_OZ.m4v")
# cap = cv2.VideoCapture("../videos/Extrait3-Vertigo-Dream_Scene(320p).m4v")
# cap = cv2.VideoCapture('../videos/Extrait1-Cosmos_Laundromat1(340p).m4v')
ret, frame1 = cap.read()  # Passe à l'image suivante

prvs = cv2.cvtColor(frame1, cv2.COLOR_BGR2GRAY)  # Passage en niveaux de gris
hsv = np.zeros_like(frame1)  # Image nulle de même taille que frame1 (affichage OF)
hsv[:, :, 1] = 255  # Toutes les couleurs sont saturées au maximum

index = 1
ret, frame2 = cap.read()
next = cv2.cvtColor(frame2, cv2.COLOR_BGR2GRAY)

while (ret):
    index += 1
    flow = cv2.calcOpticalFlowFarneback(prvs, next, None,
                                        pyr_scale=0.5,  # Taux de réduction pyramidal
                                        levels=3,  # Nombre de niveaux de la pyramide
                                        winsize=15,
                                        # Taille de fenêtre de lissage (moyenne) des coefficients polynomiaux
                                        iterations=3,  # Nb d'itérations par niveau
                                        poly_n=7,  # Taille voisinage pour approximation polynomiale
                                        poly_sigma=1.5,  # E-T Gaussienne pour calcul dérivé
                                        flags=0)

    # L'histogramme utilise -flow et [1, 0] au lieu de [0, 1] afin de faire correspondre visuellement les vitesses
    # (Points nombreux à gauche dans l'histogramme quand vitesse de l'image vers la gauche etc)
    histr = cv2.calcHist([-flow[:, :, 0], -flow[:, :, 1]], [1, 0], None, [512, 512], [-1, 1, -1, 1])

    histr = np.log(histr)/np.log(histr.max())*255
    histr[histr == -np.inf] = 0
    histr = histr.astype(np.uint8)


    mag, ang = cv2.cartToPolar(flow[:, :, 0], flow[:, :, 1])  # Conversion cartésien vers polaire
    hsv[:, :, 0] = (ang * 180) / (2 * np.pi)  # Teinte (codée sur [0..179] dans OpenCV) <--> Argument
    hsv[:, :, 2] = (mag * 255) / np.amax(mag)  # Valeur <--> Norme

    bgr = cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)
    result = np.vstack((frame2, bgr))
    cv2.imshow('Image et Champ de vitesses (Farneback)', result)

    histr = cv2.applyColorMap(histr, cv2.COLORMAP_JET)

    cv2.imshow('Histo', histr)

    k = cv2.waitKey(15) & 0xff
    if k == 27:  # escape
        break
    elif k == ord('s'):  # or index == 100:
        cv2.imwrite('Frame_%04d_%d.png' % (index, time.time()), frame2)
        cv2.imwrite('OF_hsv_%04d_%d.png' % (index, time.time()), bgr)

    prvs = next
    ret, frame2 = cap.read()
    if ret:
        next = cv2.cvtColor(frame2, cv2.COLOR_BGR2GRAY)

cap.release()
cv2.destroyAllWindows()

Après visualisation de quelques extraits, on remarque plusieurs types de scène reconnaissables par l'histogramme des vitesses :

- Plan fixe : Vitesses centrées autour de 0 avec un écart-type a priori faible
- Panoramique :
    - Horizontal : Vy autour de 0 et Vx regroupées autour d'une valeur moyenne non nulle
    - Vertical : Vx autour de 0 et Vy regroupées autour d'une valeur moyenne non nulle
- Rotation : Histogramme très homogène, pas de plage de valeur particulière de Vx et Vy -> faible entropie ?
- Travelling : Vx et Vy sont plus largement distribuées dans l'histogramme
- Zoom : Très similaire au travelling avant/arrière

Ainsi, on a déterminé quelques métriques associées à l'histogramme 2d qui pourraient être pertinentes pour comparer les types de plan :
- Valeurs max et min selon x et y
- Valeurs moyennes selon x et y
- Entropies selon x et y (faible = homogène)
- Mode (couple de vitesses majoritaire)
- Amplitude selon x et y

# Découpage et indexation

### Q4 Découpage en plans

Pour rappel, l'idée première était de calculer l'auto-corrélation entre l'histogramme courant et le précédent car on remarque des changements soudains lors des changements de plan alors que les scènes ont des changements progressifs de leur histogramme.
La valeur de cette auto-corrélation serait ensuite comparée à un seuil prédéfini et lorsque l'auto-corrélation dépasse ce seuil, on détecterait un changement de plan.

Un premier essai a été réalisé en utilisant la somme des différences absolues (SAD) pour comparer les histogrammes. Cette méthode détecte les changements de plan, mais également les changements rapides de couleur.

Comme il ne semble pas possible de définir une valeur de threshold fixe efficace l'idée est venue de calculer une valeur de threshold en fonction des n derniers histogrammes afin de s'adapter à la scène en cours.

Le threshold est calculé comme suivant : threshold = threshold_gain*(moyenne des SAD des nb_hists derniers histogrammes)

threshold_gain permet donc de contrôler la sensibilité de détection de changement de scène.

Les changements de plan sont notés par un point rouge en haut à gauche de l'image de base et un bip sonore (fonctionnel en fonction .
Voici le code :

In [5]:
import cv2
import numpy as np

# Ouverture du flux video
cap = cv2.VideoCapture("../videos/Extrait5-Matrix-Helicopter_Scene(280p).m4v")
# cap = cv2.VideoCapture("../videos/Rotation_OX(Tilt).m4v")
# cap = cv2.VideoCapture("../videos/Rotation_OY(Pan).m4v")
# cap = cv2.VideoCapture("../videos/Rotation_OZ(Roll).m4v")
# cap = cv2.VideoCapture("../videos/ZOOM_O_TRAVELLING.m4v")
# cap = cv2.VideoCapture("../videos/Travelling_OX.m4v")
# cap = cv2.VideoCapture("../videos/Travelling_OZ.m4v")
# cap = cv2.VideoCapture("../videos/Extrait3-Vertigo-Dream_Scene(320p).m4v")
# cap = cv2.VideoCapture('../videos/Extrait1-Cosmos_Laundromat1(340p).m4v')

# Convention opencv
# Si entier alors entre 0 et 255
# Si float entre 0 et 1

# Paramètres du cercle indicateur de changement de plan
center = (50, 50)  # Coordonnées (x, y) du centre du cercle
radius = 25  # Rayon du cercle
color = (0, 0, 255)  # Couleur du cercle en format BGR (rouge)
thickness = -1  # Épaisseur du cercle (-1 pour remplir le cercle)

# Check if the video file was successfully loaded
if not cap.isOpened():
    print("Error loading video file")

nb_hists = 5
hist_idx = 0  # Keep track of idx of hist to add the last one in the array of hists with minimal computation
threshold_gain = 4.5
hists_array = np.zeros(shape=(nb_hists, 256, 256, 3))
SAD = 0

hist_idx = 0
# Loop through the video frames
while cap.isOpened():
    # Read the next frame from the video file
    ret, frame = cap.read()

    frameYuv = cv2.cvtColor(frame, cv2.COLOR_BGR2YUV)

    # Don't forget the [] around the frame
    hist = cv2.calcHist([frameYuv], [1, 2], None, [256, 256], [0, 255, 0, 255])
    # Log
    hist = np.log(hist)
    # Normalization
    hist = hist / hist.max() * 255
    hist = hist.astype(np.uint8)
    hist = cv2.applyColorMap(hist, cv2.COLORMAP_JET)

    # Sum of Absolute Differencies of the current histogram
    SAD = np.sum(np.abs(hist - hists_array[hist_idx - 1]))

    # init threshold to 0
    threshold = 0

    # Sum of Absolute Differencies to compare the current and previous histograms
    for i in range(1, nb_hists):
        threshold += np.sum(np.abs(hists_array[i] - hists_array[i - 1]))
    # Adjustment of the threshold with threshold_gain
    threshold = threshold_gain * threshold / nb_hists

    if SAD > threshold:
        print('\a')  # Fait un bip sonore
        frame = cv2.circle(frame, center, radius, color, thickness)
        print("Changement de plan       SAD : ", SAD, "      threshold", threshold, "        SAD/threshold",
              SAD / threshold)

    # Add the current histogram to the list for the upcoming histogram of the next iteration
    hists_array[hist_idx] = hist
    # Update the hist_idx to access the latest histogram
    hist_idx = hist_idx + 1 if (hist_idx < nb_hists - 1) else 0

    # Check if there are no more frames
    if not ret:
        break

    # Display the frame
    cv2.imshow('frame', frame)
    cv2.imshow('histogramme', hist)

    # Wait for a key press
    if cv2.waitKey(25) & 0xFF == ord('q'):
        break

# Release the video file and close the window
cap.release()
cv2.destroyAllWindows()



Changement de plan       SAD :  8453098.0       threshold 0.0         SAD/threshold inf

Changement de plan       SAD :  326479.0       threshold 193780.8         SAD/threshold 1.6847850767465096

Changement de plan       SAD :  370591.0       threshold 238576.5         SAD/threshold 1.5533424289483666


On a commencé les essais à threshold_gain = 2, ce qui a donné :
- L'amélioration la plus flagrante : les changements abondants et progressifs (bien que rapides) ne sont plus détectés. L'exemple le plus marquant est l'apparition dans le ciel de la tornade de couleur.
- Certains des éclairs de la scène sont souvent détectés comme des changements de scène. C'est logique car tous les pixels deviennent blancs donc la colorimétrie de la scène change brutalement et c'est ce que détecte le code créé.

Cependant, après quelques ajustements pour trouver le bon threshold_gain, le programme détecte la grande majorité des changements de plan mais toujours environ la moitié des éclairs dans l'extrait *Cosmos Laundromat*.

Voici certains problèmes liés à cette méthode :
- Les transitions trop douces ne sont pas correctement détectées, comme dans l'extrait *Vertigo Dream Scene*.
- Certains changements trop soudains dans la colorimétrie peuvent être détectés comme changement de plan

En niveaux de gris, on peut utiliser un histogramme 1D et réaliser exactement le même traitement. Cela a été réalisé sur l'extrait 2 *Man With A Movie Camera* et a fonctionné de façon très correcte.


### Q5 Identification de la frame principale

Afin de rester dans l'esprit des histogrammes, notre première idée est d'identifier la frame décrivant le mieux la scène -que l'on appellera la frame principale- à l'aide des maxima de l'entropie sur un plan car l'entropie permet de mesurer l'homogénéité de la répartition des vitesses dans l'image.

Après des tests sur quelques extraits, la frame identifiée par le max de l'entropie sur un plan semblait plus pertinente la majorité du temps. C'est donc cette méthode qui a été retenue.

Elle n'est cependant pas parfaite car elle tient uniquement compte des vitesses et de leur homogénéité donc l'image-clef extraite se base uniquement sur le mouvement or on comprend bien que c'est loin d'être l'unique caractéristique d'une image. L'objectif pour correctement identifier une image-clef serait de pouvoir extraire du contexte à partir de la vidéo.

### Q6 Identification de type de plan

Initialement, l'idée était de comparer les métriques déterminées en Q3 entre elles afin de déterminer le type de plan. Par exemple, si ```(Vx_mean/Vy_mean) >= threshold``` alors on a un certain type de plan et ainsi de suite pour chaque type de plan. Cela a donné l'ébauche horrifique suivante :
&nbsp;

```python
    # Plan fixe
    if Vx_abs_max <= thr_vabs_fixe and Vy_abs_max <= thr_vabs_fixe and Vx_mean <= thr_mean_fixe and Vy_mean <= thr_mean_fixe:
        return 'plan fixe'
    # Zoom et travelling avant/arrière
    elif Vx_amplitude/Vy_amplitude<=rapport_amplitudes_zoom and Vx_mean/Vy_mean <= rapport_mean_zoom:
        return 'zoom' #TODO déterminer avant ou arrière

    # Travelling horizontal
    elif np.abs(Vx_mean/Vy_mean) >= rapport_mean_trav_h:
        return 'travelling horizontal'
    elif np.abs(Vx_mean/Vy_mean) >= rapport_mean_trav_v:
        return 'travelling vertical'
    return 'indéterminé'
```
&nbsp;

Regarder chaque vidéo et trouver des relations entre les métriques uniquement vérifiées dans un type de plan précis se révéla assez pénible et peu fructueux.
J'ai donc voulu appliquer ce que j'ai appris en cours de machine learning et de tenter d'utiliser un Support Vector Classifier ou un k-NN Classifier avec Scikit Learn.

Afin de trouver le meilleur modèle parmi SVC ou k-NN avec les meilleurs paramètres, j'ai utilisé GridSearch puis sauvegardé le modèle avec la bibliothèque ```joblib```.
Le code complet se trouve dans ```Grid-Search-Svm-Or-Knn.py```.

Les données d'entrainement ont été générées avec ```Data-Maker.py```. Cette version modifiée de ```Dense-Optical-Flow.py``` permet d'annoter l'image courante via les touches du keypad (un numéro par classe) et d'enregistrer les données dans ```data.csv```.

Finalement le classifieur k-NN était le plus performant d'après plusieurs tests. Voici les paramètres et une moyenne des performances :
```
Meilleurs hyperparamètres k-NN : {'metric': 'manhattan', 'n_neighbors': 3, 'weights': 'uniform'}
Meilleur score de validation croisée k-NN : 0.7036363636363636
Score sur l'ensemble de test : 0.75
```

Voici le code permettant d'identifer le type de plan :


In [6]:
import time
import cv2
import numpy as np
import pandas as pd
import joblib
import Metrics

np.seterr(divide='ignore', invalid='ignore')
# Chargement du k-NN entrainé
model_path = '../models/knn_plan.joblib'
knn = joblib.load(model_path)
type_plans = ['plan fixe', 'pano horizontal', 'pano vertical', 'rotation', 'trav horizontal', 'trav vertical',
              'trav avant', 'trav arriere', 'zoom avant', 'zoom arriere']
X_columns = ["Vx_entropy", "Vy_entropy", "Vx_amplitude", "Vy_amplitude", "Vx_max", "Vy_max", "Vx_min", "Vy_min",
             "Vx_mean", "Vy_mean"]

# Paramètres pour affichage du type de plan
font = cv2.FONT_HERSHEY_SIMPLEX  # Police du texte
font_scale = 1  # Échelle de la police (taille du texte)
color = (255, 255, 255)  # Couleur du texte (B, G, R)
thickness = 2  # Épaisseur des lignes du texte

# Ouverture du flux video
# cap = cv2.VideoCapture("../videos/Extrait5-Matrix-Helicopter_Scene(280p).m4v")
cap = cv2.VideoCapture("../videos/Rotation_OX(Tilt).m4v")
# cap = cv2.VideoCapture("../videos/Rotation_OY(Pan).m4v")
# cap = cv2.VideoCapture("../videos/Rotation_OZ(Roll).m4v")
# cap = cv2.VideoCapture("../videos/ZOOM_O_TRAVELLING.m4v")
# cap = cv2.VideoCapture("../videos/Travelling_OX.m4v")
# cap = cv2.VideoCapture("../videos/Travelling_OZ.m4v")
# cap = cv2.VideoCapture("../videos/Extrait3-Vertigo-Dream_Scene(320p).m4v")
# cap = cv2.VideoCapture('../videos/Extrait1-Cosmos_Laundromat1(340p).m4v')

ret, frame1 = cap.read()  # Passe à l'image suivante

if frame1 is None:
    print("Erreur")

prvs = cv2.cvtColor(frame1, cv2.COLOR_BGR2GRAY)  # Passage en niveaux de gris
hsv = np.zeros_like(frame1)  # Image nulle de même taille que frame1 (affichage OF)
hsv[:, :, 1] = 255  # Toutes les couleurs sont saturées au maximum
index = 1
ret, frame2 = cap.read()
next = cv2.cvtColor(frame2, cv2.COLOR_BGR2GRAY)

while (ret):
    index += 1

    # Renvoie numpy.ndarray a la même taille que les images d'entrée et contient les coordonnées du vecteur
    # de mouvement (u, v) pour chaque pixel. Les coordonnées du vecteur sont stockées sous forme de canaux,
    # où le premier canal (u) représente le déplacement horizontal et le deuxième canal (v) représente le déplacement vertical
    flow = cv2.calcOpticalFlowFarneback(prvs, next, None,
                                        pyr_scale=0.5,  # Taux de réduction pyramidal
                                        levels=5,  # Nombre de niveaux de la pyramide
                                        winsize=15,
                                        # Taille de fenêtre de lissage (moyenne) des coefficients polynomiaux
                                        iterations=3,  # Nb d'itérations par niveau
                                        poly_n=5,  # Taille voisinage pour approximation polynomiale
                                        poly_sigma=0.5,  # E-T Gaussienne pour calcul dérivé
                                        flags=0)

    # L'histogramme utilise flow et [1, 0] au lieu de [0, 1] afin de faire correspondre visuellement les vitesses
    # (Points nombreux à gauche dans l'histogramme quand vitesse de l'image négative etc)
    histr = cv2.calcHist([flow[:, :, 0], flow[:, :, 1]], [1, 0], None, [512, 512], [-1, 1, -1, 1])

    X = Metrics.get_X_vector(flow, histr)  # Calcul à partir de l'histogramme et du flow par facilité des formules
    X = pd.DataFrame(data=[X], columns=X_columns)
    type_plan = type_plans[knn.predict(X)[0]] if not X.isna().any().any() else " "

    histr = np.log(histr) / np.log(histr.max()) * 255
    histr[histr == -np.inf] = 0
    histr = histr.astype(np.uint8)

    mag, ang = cv2.cartToPolar(flow[:, :, 0], flow[:, :, 1])  # Conversion cartésien vers polaire
    hsv[:, :, 0] = (ang * 180) / (2 * np.pi)  # Teinte (codée sur [0..179] dans OpenCV) <--> Argument
    hsv[:, :, 2] = (mag * 255) / np.amax(mag)  # Valeur <--> Norme

    bgr = cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)
    result = np.vstack((frame2, bgr))
    result = cv2.putText(result, type_plan, org=(frame1.shape[0] // 2, frame1.shape[1] // 2 + 50), fontFace=font,
                         fontScale=font_scale, color=color, thickness=thickness)

    histr = cv2.applyColorMap(histr, cv2.COLORMAP_JET)

    cv2.imshow('Histo', histr)
    cv2.imshow('Image et Champ de vitesses (Farneback)', result)

    k = cv2.waitKey(15) & 0xff
    if k == 27:  # escape
        break
    elif k == ord('s'):  # or index == 100:
        cv2.imwrite('Frame_%04d_%d.png' % (index, time.time()), frame2)
        cv2.imwrite('OF_hsv_%04d_%d.png' % (index, time.time()), bgr)

    prvs = next
    ret, frame2 = cap.read()
    if ret:
        next = cv2.cvtColor(frame2, cv2.COLOR_BGR2GRAY)

cap.release()
cv2.destroyAllWindows()

À noter que cette méthode estime le type de plan pour chaque frame or on veut identifier le type de plan sur un plan entier détecté automatiquement.
Ainsi, un système de votes par frame est mis en place et le type de plan identifié sera celui avec le plus de votes sur un plan donné.
&nbsp;

Finalement, on peut émettre quelques remarques :
- La technique marche correctement même si cela est probablement biaisé par le fait que les données d'entraînement sont issues des données d'utilisation finales.
    - Le dataset de train est très petit (77 vecteurs) car c'est plutôt la mise en oeuvre qui m'importait dans ce TP
- Le modèle confond souvent zoom avant (et arrière) et travelling avant (et arrière) lorsqu'on estime le type de plan sur une image du flot. Cependant, le système de vote majoritaire corrige ceci et le modèle ne confond pas ces types de plan sur un plan entier.


# La combinaison des méthodes montrées jusqu'ici est essayable en exécutant la cellule suivante :
&nbsp;
### Le script python va déterminer et afficher une image descriptive pour chaque plan détecté de l'extrait choisi. L'image sera affichée dans une fenêtre à part en-dessous de l'image potentiellement principale du plan en cours.

In [7]:
# Au cas où vous n'auriez pas exécuté les cellules nécessaires
import os
# os.chdir("src")

''' Pour essayer avec différentes vidéos
"Extrait5-Matrix-Helicopter_Scene(280p).m4v"
"Rotation_OX(Tilt).m4v"
"Rotation_OY(Pan).m4v"
"Rotation_OZ(Roll).m4v"
"ZOOM_O_TRAVELLING.m4v"
"Travelling_OX.m4v"
"Travelling_OZ.m4v"
"videos/Extrait3-Vertigo-Dream_Scene(320p).m4v"
"videos/Extrait1-Cosmos_Laundromat1(340p).m4v"
'''

file_name = 'ZOOM_O_TRAVELLING.m4v'
DISPLAY_HISTOGRAMS = 0 # 0 to disable display of uv and Vx Vy histograms
DISPLAY_FLOW = 1 # 0 to disable display of flow
DISPLAY_MAIN_FRAME = 1 # 0 to disable display of the main frame of the shot
SAVE_MAIN_FRAMES = 0 # 0 to disable the saving of the main frames as .png

print(os.getcwd())
os.system(f"python3 Scene-Splitter.py '{file_name}' {DISPLAY_HISTOGRAMS} {DISPLAY_FLOW} {DISPLAY_MAIN_FRAME} {SAVE_MAIN_FRAMES}")


/home/wnbilly/Cours/mi204/OpticalFlow/src





0